# COLAB TPU 2.8 training

In [ ]:
################## EFF_NETS = [ ？ ]*FOLDS #######################
#########挂载硬盘上训练，每折都保存最优模型，N_TRAINED_FOLD>0为接着训练##########

In [ ]:
######## 已经训练完的折数########
N_TRAINED_FOLD = 0
USE_TRAINED_MODEL = True # False = 测试N_TRAINED_FOLD后的步骤

# 快速测试跑通一遍
FAST_KAGGING = False

################可视化################
# VERBOSE = 0
# DISPLAY_PLOT = False # 显示训练图
VERBOSE = 1
DISPLAY_PLOT = True # 显示训练图
DISPLAY_TRAINED_PLOT = True # 显示已经训练的model图

In [ ]:
# Colab google
from google.colab import drive
drive.mount('/content/drive')
# 运行目录
import os
os.chdir("/content/drive/My Drive/siim") ######### My Drive ##########

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import time
start = time.time()

In [ ]:
!pip install tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0
import tensorflow as tf
import requests
import os
resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
if resp.status_code != 200:
  print("Failed to switch the TPU to TF {}".format(version))

####################################
# colab pro这段时间内存报错的问题，加一段代码就可以了 from JWD
# https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/172357，
####################################

     |████████████████████████████████| 516.2MB 16kB/s 
     |████████████████████████████████| 399kB 35.7MB/s 
     |████████████████████████████████| 460kB 61.1MB/s 
     |████████████████████████████████| 3.0MB 54.6MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
  Found existing installation: tensorflow-gcs-config 2.3.0
    Uninstalling tensorflow-gcs-config-2.3.0:
      Successfully uninstalled tensorflow-gcs-config-2.3.0


In [ ]:
############### data if kagging ###################
KAGGING = False # 
SEED_TEST = list(range(2021)) # KAGGING_TEST RANGE
KAGGING_SEED = 24
print(SEED_TEST[-3:])


[2018, 2019, 2020]


In [ ]:
!pip install -q efficientnet 

     |████████████████████████████████| 51kB 2.8MB/s eta 0:00:011


In [ ]:
import pandas as pd, numpy as np
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
DEVICE = "TPU" # "GPU" 


# SEED = 42
if KAGGING: SEEDS = [42]
else: SEEDS = [1911, 2822, 3733, 4644, 55055, 6466, 7377, 8288, 9199]


# FOLDS = 5
FOLDS = 2 if FAST_KAGGING else 5 # 2

#  384, 512, 768 image size
# IMG_SIZES = [ 512 ] * FOLDS
IMG_SIZES = [ 384 ]*FOLDS if FAST_KAGGING else [ 384 ]*FOLDS

# bacth size
BATCH_SIZES = [ 32 ]*FOLDS if FAST_KAGGING else [ 8 ]*FOLDS # 32太大了，16 >> B6-512-15bs


EPOCHS = [ 2 ]*FOLDS if FAST_KAGGING else [ 20 ]*FOLDS



# EFF_NETS = [ 6 ]*FOLDS
EFF_NETS = [ 6 ]*FOLDS if FAST_KAGGING else [ 6 ]*FOLDS



# WEIGHTS FOR FOLD MODELS WHEN PREDICTING TEST
WGTS = [1/FOLDS]*FOLDS
# WGTS = [.25,.25,0,.25,.25] ##temp## ########调整各个fold的pred的权重（根据Local CV）###########


############差异化融合###################
# (2018/2017) 和 2019数据集
INC2019 = [  1 ] * FOLDS
INC2018 = [  1 ] * FOLDS
# INC2019 = [  0 ] * FOLDS
# INC2018 = [  0 ] * FOLDS

############差异化融合###################
# 由于本次只有2%的得病，进行上采样
# 上采样malig
# 如果2018和2019外源数据都用了，只会加500张没见过的
# 如果只用2018，则加2019和500张没见过的
# 如果只用2019，则加2018和500张没见过的
# 如果都没用，则加2018，2019和500张没见过的
MALIG = [ 1 ]*FOLDS
# MALIG = [ 0 ]*FOLDS


# TEST TIME AUGMENTATION STEPS，10+ steps
# TTA = 10
TTA = 13
# 标签平滑，可以尝试0.015，抗过拟合
# ls = 0.02
ls = 0.015


# 训练schedule
def get_lr_callback(batch_size=8):
    #############需细致调整###################
    # lr_start   = 1e-6 # 初始学习率
    # lr_max   =  2e-4# 最大学习率
    # lr_min     = 1e-7 #最小学习率
    # lr_ramp_ep =  10 # 用几个epoch达到最大学习率
    # lr_sus_ep  =  2# 用最大的学习率跑几个epoch
    # lr_decay   = .5 # 退火
    ################## 8月 #####################
    # lr_start   = 5e-6 # 初始学习率
    # lr_max   =  1e-4# 最大学习率
    # lr_min     = 1e-6 #最小学习率
    # lr_ramp_ep =  14 # 用几个epoch达到最大学习率
    # lr_sus_ep  =  1 # 用最大的学习率跑几个epoch
    # lr_decay   = .4 # 退火
    lr_start   = 1e-5 # 初始学习率
    lr_max   =  1e-4# 最大学习率
    lr_min     = 1e-6 #最小学习率
    lr_ramp_ep =  20 # 用几个epoch达到最大学习率
    lr_sus_ep  =  1 # 用最大的学习率跑几个epoch
    lr_decay   = .4 # 退火

    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

# 每一个epoch的情况
# VERBOSE = 1
# DISPLAY_PLOT = True

SEED = KAGGING_SEED


print('seed:{}'.format(SEED))

SIIM_para = {}

# SIIM_para['SEED'] = SEED
SIIM_para['FOLDS'] = FOLDS
SIIM_para['IMG_SIZES'] = IMG_SIZES
SIIM_para['TTA'] = TTA
SIIM_para['ls'] = ls
SIIM_para['INC2019'] = INC2019
SIIM_para['INC2018'] = INC2018
SIIM_para['BATCH_SIZES'] = BATCH_SIZES
SIIM_para['EPOCHS'] = EPOCHS
SIIM_para['EFF_NETS'] = EFF_NETS
SIIM_para['MALIG'] = MALIG


print('SIIM_parameters: {}'.format(SIIM_para))

Team4 seed:550550
SIIM_parameters: {'FOLDS': 5, 'IMG_SIZES': [384, 384, 384, 384, 384], 'TTA': 13, 'ls': 0.015, 'INC2019': [1, 1, 1, 1, 1], 'INC2018': [1, 1, 1, 1, 1], 'BATCH_SIZES': [8, 8, 8, 8, 8], 'EPOCHS': [20, 20, 20, 20, 20], 'EFF_NETS': [6, 6, 6, 6, 6], 'MALIG': [1, 1, 1, 1, 1]}


In [ ]:
assert (IMG_SIZES==[ 384 ]*FOLDS) & (EFF_NETS == [ 6 ]*FOLDS)

In [ ]:
if not KAGGING:
    assert SEED not in SEED_TEST+[42]
else:
    assert SEED not in SEEDS+[42]

In [ ]:
# 使用tpu
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

connecting to TPU...
Running on TPU  grpc://10.11.163.106:8470
initializing  TPU ...
INFO:tensorflow:Initializing the TPU system: grpc://10.11.163.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.11.163.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU initialized
REPLICAS: 8


In [ ]:
# #### UnimplementedError TPU不支持local load，只能通过google storage ######
# GCS_PATH = ['./input/melanoma-{}x{}'.format(IMG_SIZES[0],IMG_SIZES[0])]*FOLDS 
# GCS_PATH2 = ['./input/isic2019-{}x{}'.format(IMG_SIZES[0],IMG_SIZES[0])]*FOLDS
# GCS_PATH3 = ['./input/malignant-v2-{}x{}'.format(IMG_SIZES[0],IMG_SIZES[0])]*FOLDS

if IMG_SIZES[0] == 256:
  GCS_PATH = ['gs://kds-f8af4bf2f6c765dbd327683a170e364332a1750658e910153e98d0d5']*FOLDS
  GCS_PATH2 = ['gs://kds-436e76b7e255e4499d5d11da9d7ab87646e271e08ec1d546f310ec00']*FOLDS
  GCS_PATH3 = ['gs://kds-9e5912113bdca38bda75e6858a127bdfd0f26951b4f6843c45974106']*FOLDS
elif IMG_SIZES[0] == 384: 
  GCS_PATH = ['gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4']*FOLDS
  GCS_PATH2 = ['gs://kds-6f9d2143e7fe97eccac0219c1aacb107a234b440670f250683ba40ee']*FOLDS
  GCS_PATH3 = ['gs://kds-c557c41c0102dc29c1ed0386c4746604d132a95f29eeacf42e522004']*FOLDS
elif IMG_SIZES[0] == 512:
  GCS_PATH = ['gs://kds-458e4ec4a30e5d927b21efd0633fb2c0a091245fe8db3c415fde5da6']*FOLDS
  GCS_PATH2 = ['gs://kds-203bea6cc14c2866536ac303b09a6e77a6b3d0a371e96b680836433c']*FOLDS
  GCS_PATH3 = ['gs://kds-00457c0c555bfdd50b1d2db15abd7feb19d4795037f919a3dc0abbee']*FOLDS
elif IMG_SIZES[0] == 768:
  GCS_PATH = ['gs://kds-cfb16ed5f55adb5ab35a75a2fe74c3dc11a4b869dc8cfb3d9b1759e1']*FOLDS
  GCS_PATH2 = ['gs://kds-79d9c56dcae7978f98a6bbe8318ac6866ba4778fc95c5871fcfd0f03']*FOLDS
  GCS_PATH3 = ['gs://kds-032edf427ad6e338f5392f73759ff404a1a3f25c2abdc4995136de30']*FOLDS
else:
  print('Please choose a value between 384, 512 or 768')

files_train = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/train*.tfrec')))
files_test  = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[0] + '/test*.tfrec')))


In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0


EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]


def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))

# transform augmentation by chris
def transform(image, DIM=256):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

# 读取tfrecord
def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
        'patient_id'                   : tf.io.FixedLenFeature([], tf.int64),
        'sex'                          : tf.io.FixedLenFeature([], tf.int64),
        'age_approx'                   : tf.io.FixedLenFeature([], tf.int64),
        'anatom_site_general_challenge': tf.io.FixedLenFeature([], tf.int64),
        'diagnosis'                    : tf.io.FixedLenFeature([], tf.int64),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']

# 读取test set的tfrecord
def read_unlabeled_tfrecord(example, return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name'] if return_image_name else 0

 # 图片
def prepare_image(img, augment=True, dim=256):    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    
    if augment:
        img = transform(img,DIM=dim)
        img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_hue(img, 0.01)
        # img = tf.image.random_saturation(img, 0.7, 1.3)
        # img = tf.image.random_contrast(img, 0.8, 1.2)
        # img = tf.image.random_brightness(img, 0.1)
        # 没啥用
                      
    img = tf.reshape(img, [dim,dim, 3])
            
    return img

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)


def get_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True, return_image_names=True, batch_size=16, dim=256):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_names), num_parallel_calls=AUTO)      
    # ###########################数据增强############################
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, dim=dim), imgname_or_label), num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds



def build_model(dim=128, ef=0):
    
    inp = tf.keras.layers.Input(shape=(dim,dim,3))
    base = EFNS[ef](input_shape=(dim,dim,3),weights='imagenet',include_top=False)
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=ls) 
    model.compile(optimizer=opt,loss=loss,metrics=['AUC'])
    
    return model

In [ ]:
oof_pred = []; oof_tar = []; oof_val = []; oof_names = []; oof_folds = [] 
preds = np.zeros((count_data_items(files_test),1))
skf = KFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
# Train_val_split
for fold,(idxT,idxV) in enumerate(skf.split(np.arange(15))):   
    print('Range', fold, (idxT,idxV))
    #############未训练的###################
    # if fold >= N_TRAINED_FOLD:
    # DISPLAY FOLD INFO
    if DEVICE=='TPU':
        if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print('#'*25); print('#### FOLD',fold+1)
    print('#### Image Size %i with EfficientNet B%i and batch_size %i'%
        (IMG_SIZES[fold],EFF_NETS[fold],BATCH_SIZES[fold]*REPLICAS))
    
    # CREATE TRAIN AND VALIDATION SUBSETS
    files_train = tf.io.gfile.glob([GCS_PATH[fold] + '/train%.2i*.tfrec'%x for x in idxT])
    if INC2019[fold]:
        files_train += tf.io.gfile.glob([GCS_PATH2[fold] + '/train%.2i*.tfrec'%x for x in idxT*2+1])
        print('#### Using 2019 external data')
        print(files_train)
    if INC2018[fold]:
        files_train += tf.io.gfile.glob([GCS_PATH2[fold] + '/train%.2i*.tfrec'%x for x in idxT*2])
        print('#### Using 2018+2017 external data')
        # print(files_train)
 
    # malignant upsample
    if MALIG[fold]:
        if INC2018[fold] and INC2019[fold]:
            malig_idx = [i for i in range(15,30)] # 15 - 30 unseen
            files_train += tf.io.gfile.glob([GCS_PATH3[fold] + '/train%.2i*.tfrec'%x for x in malig_idx])
            print('#### Using unseen extra malignant data!!!!!!!!!!')
            # print(files_train)
        elif INC2018[fold]:
            malig_idx = [i for i in range(15,30)] + [j for j in range(31, 60,2)]  # 15 - 30 unseen + 31 - 59 odd
            files_train += tf.io.gfile.glob([GCS_PATH3[fold] + '/train%.2i*.tfrec'%x for x in malig_idx])
            print('#### Using 2019 + unseen extra malignant data!!!!!!!!!!')
            # print(files_train)
        elif INC2019[fold]:
            malig_idx = [i for i in range(15,30)] + [j for j in range(30, 60,2)]  # 15 - 30 unseen + 30 - 58 even
            files_train += tf.io.gfile.glob([GCS_PATH3[fold] + '/train%.2i*.tfrec'%x for x in malig_idx])
            print('#### Using 2018 + unseen extra malignant data!!!!!!!!!!')
            # print(files_train)
        else:
            malig_idx = [i for i in range(15,60)] # 15 - 59 this includes all 2018 and 2019
            files_train += tf.io.gfile.glob([GCS_PATH3[fold] + '/train%.2i*.tfrec'%x for x in malig_idx])
            print('#### Using all extra malignant data!!!!!!!!!!')


    np.random.shuffle(files_train); print('#'*25)
    files_valid = tf.io.gfile.glob([GCS_PATH[fold] + '/train%.2i*.tfrec'%x for x in idxV])
    print(files_valid)
    files_test = np.sort(np.array(tf.io.gfile.glob(GCS_PATH[fold] + '/test*.tfrec')))
    
    # BUILD MODEL
    K.clear_session()
    with strategy.scope():
        model = build_model(dim=IMG_SIZES[fold],ef=EFF_NETS[fold])
        
    # SAVE BEST MODEL EACH FOLD
    sv = tf.keras.callbacks.ModelCheckpoint('fold-%i.h5'%(fold+1), monitor='val_auc', verbose=0, save_best_only=True, save_weights_only=True, mode='max', save_freq='epoch')

    ##############未训练的###################
    if fold >= N_TRAINED_FOLD:
        # TRAIN
        print('Training...')
        history = model.fit(
            get_dataset(files_train, augment=True, shuffle=True, repeat=True, dim=IMG_SIZES[fold], batch_size = BATCH_SIZES[fold]), 
            epochs=EPOCHS[fold], callbacks = [sv,get_lr_callback(BATCH_SIZES[fold])], 
            steps_per_epoch=count_data_items(files_train)/BATCH_SIZES[fold]//REPLICAS,
            validation_data=get_dataset(files_valid,augment=False,shuffle=False, repeat=False,dim=IMG_SIZES[fold]), #class_weight = {0:1,1:2},
            verbose=VERBOSE,
        )

        # save history pickle
        print('Saving train history...') 
        import pickle
        with open('hist-fold-%i.pickle'%(fold+1), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

        # print('Loading best model fold-%i.h5...'%(fold+1))
        # model.load_weights('fold-%i.h5'%(fold+1))

        print('Loading best model fold-%i.h5...'%(fold+1))
        model.load_weights('fold-%i.h5'%(fold+1))
        # PREDICT OOF USING TTA
        print('Predicting OOF with TTA...')
        ds_valid = get_dataset(files_valid,labeled=False,return_image_names=False,augment=True, repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
        ct_valid = count_data_items(files_valid)
        STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
        pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
        oof_pred.append(np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )                 
        #oof_pred.append(model.predict(get_dataset(files_valid,dim=IMG_SIZES[fold]),verbose=1))
        
        # GET OOF TARGETS AND NAMES
        ds_valid = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=True, return_image_names=True)
        oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )
        oof_folds.append( np.ones_like(oof_tar[-1],dtype='int8')*fold )
        ds = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=False, return_image_names=True)
        oof_names.append( np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())]))
        
        # PREDICT TEST USING TTA
        print('Predicting Test with TTA...')
        ds_test = get_dataset(files_test,labeled=False,return_image_names=False,augment=True,
                repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
        ct_test = count_data_items(files_test)
        STEPS = TTA * ct_test/BATCH_SIZES[fold]/4/REPLICAS
        pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,] 


        #########Data if Kagging###########
        # save temp pred
        pred_temp = np.zeros((count_data_items(files_test),1))
        pred_temp[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1)

        ds = get_dataset(files_test, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=False, return_image_names=True)
        image_names = np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())])

        submission_temp = pd.DataFrame(dict(image_name=image_names, target=pred_temp[:,0]))
        submission_temp = submission_temp.sort_values('image_name') 
        submission_temp.to_csv('submission_temp_fold{}.csv'.format(fold+1), index=False)
        # pred_temp.to_csv('pred_temp_fold{}'.format(fold+1))
        #######################


        preds[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1) * WGTS[fold]    
        # REPORT RESULTS
        auc = roc_auc_score(oof_tar[-1],oof_pred[-1])

        oof_val.append(np.max( history.history['val_auc'] ))
        print('#### FOLD %i OOF AUC without TTA = %.3f, with TTA = %.3f'%(fold+1,oof_val[-1],auc))
        # 2
        if DISPLAY_PLOT:
            plt.figure(figsize=(15,5))
            plt.plot(np.arange(EPOCHS[fold]),history.history['auc'],'-o',label='Train AUC',color='#ff7f0e')
            plt.plot(np.arange(EPOCHS[fold]),history.history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
            x = np.argmax( history.history['val_auc'] ); y = np.max( history.history['val_auc'] )
            xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
            plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
            plt.ylabel('AUC',size=14); plt.xlabel('Epoch',size=14)
            plt.legend(loc=2)
            plt2 = plt.gca().twinx()
            plt2.plot(np.arange(EPOCHS[fold]),history.history['loss'],'-o',label='Train Loss',color='#2ca02c')
            plt2.plot(np.arange(EPOCHS[fold]),history.history['val_loss'],'-o',label='Val Loss',color='#d62728')
            x = np.argmin( history.history['val_loss'] ); y = np.min( history.history['val_loss'] )
            ydist = plt.ylim()[1] - plt.ylim()[0]
            plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
            plt.ylabel('Loss',size=14)
            plt.title('FOLD %i - Image Size %i, EfficientNet B%i, inc2019=%i, inc2018=%i'%
                    (fold+1,IMG_SIZES[fold],EFF_NETS[fold],INC2019[fold],INC2018[fold]),size=18)
            plt.legend(loc=3)
            plt.show()  

    else:
        ############## 已经训练的 ###################
        print('>>> Fold-%i was trained...'%(fold+1))
        if USE_TRAINED_MODEL:
            
            print('Use trained model >>>')
            # print('Loading best model fold-%i.h5...'%(fold+1))
            # model.load_weights('fold-%i.h5'%(fold+1))

            # ##############
            # # BUILD MODEL
            # K.clear_session()
            # with strategy.scope():
            #     model = build_model(dim=IMG_SIZES[fold],ef=EFF_NETS[fold])
            # ##############

            # load history pickle
            print('Loading train history...') 
            import pickle
            with open('hist-fold-%i.pickle'%(fold+1), 'rb') as f:
                history = pickle.load(f)        


            print('Loading best model fold-%i.h5...'%(fold+1))
            model.load_weights('fold-%i.h5'%(fold+1))
            # PREDICT OOF USING TTA
            print('Predicting OOF with TTA...')
            ds_valid = get_dataset(files_valid,labeled=False,return_image_names=False,augment=True, repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
            ct_valid = count_data_items(files_valid)
            STEPS = TTA * ct_valid/BATCH_SIZES[fold]/4/REPLICAS
            pred = model.predict(ds_valid,steps=STEPS,verbose=VERBOSE)[:TTA*ct_valid,] 
            oof_pred.append(np.mean(pred.reshape((ct_valid,TTA),order='F'),axis=1) )                 
            #oof_pred.append(model.predict(get_dataset(files_valid,dim=IMG_SIZES[fold]),verbose=1))
            
            # GET OOF TARGETS AND NAMES
            ds_valid = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=True, return_image_names=True)
            oof_tar.append( np.array([target.numpy() for img, target in iter(ds_valid.unbatch())]) )
            oof_folds.append( np.ones_like(oof_tar[-1],dtype='int8')*fold )
            ds = get_dataset(files_valid, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=False, return_image_names=True)
            oof_names.append( np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())]))
            
            # PREDICT TEST USING TTA
            print('Predicting Test with TTA...')
            ds_test = get_dataset(files_test,labeled=False,return_image_names=False,augment=True,
                    repeat=True,shuffle=False,dim=IMG_SIZES[fold],batch_size=BATCH_SIZES[fold]*4)
            ct_test = count_data_items(files_test)
            STEPS = TTA * ct_test/BATCH_SIZES[fold]/4/REPLICAS
            pred = model.predict(ds_test,steps=STEPS,verbose=VERBOSE)[:TTA*ct_test,] 


            #########Data if Kagging###########
            # save temp pred
            pred_temp = np.zeros((count_data_items(files_test),1))
            pred_temp[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1)

            ds = get_dataset(files_test, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=False, return_image_names=True)
            image_names = np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())])

            submission_temp = pd.DataFrame(dict(image_name=image_names, target=pred_temp[:,0]))
            submission_temp = submission_temp.sort_values('image_name') 
            submission_temp.to_csv('submission_temp_fold{}.csv'.format(fold+1), index=False)
            # pred_temp.to_csv('pred_temp_fold{}'.format(fold+1))
            #######################


            preds[:,0] += np.mean(pred.reshape((ct_test,TTA),order='F'),axis=1) * WGTS[fold]    
            # REPORT RESULTS
            auc = roc_auc_score(oof_tar[-1],oof_pred[-1])

            oof_val.append(np.max( history['val_auc'] ))
            print('#### FOLD %i OOF AUC without TTA = %.3f, with TTA = %.3f'%(fold+1,oof_val[-1],auc))

            # PLOT TRAINING
            # if DISPLAY_PLOT:
            if DISPLAY_PLOT and DISPLAY_TRAINED_PLOT:
                plt.figure(figsize=(15,5))
                plt.plot(np.arange(EPOCHS[fold]),history['auc'],'-o',label='Train AUC',color='#ff7f0e')
                plt.plot(np.arange(EPOCHS[fold]),history['val_auc'],'-o',label='Val AUC',color='#1f77b4')
                x = np.argmax( history['val_auc'] ); y = np.max( history['val_auc'] )
                xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
                plt.scatter(x,y,s=200,color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max auc\n%.2f'%y,size=14)
                plt.ylabel('AUC',size=14); plt.xlabel('Epoch',size=14)
                plt.legend(loc=2)
                plt2 = plt.gca().twinx()
                plt2.plot(np.arange(EPOCHS[fold]),history['loss'],'-o',label='Train Loss',color='#2ca02c')
                plt2.plot(np.arange(EPOCHS[fold]),history['val_loss'],'-o',label='Val Loss',color='#d62728')
                x = np.argmin( history['val_loss'] ); y = np.min( history['val_loss'] )
                ydist = plt.ylim()[1] - plt.ylim()[0]
                plt.scatter(x,y,s=200,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
                plt.ylabel('Loss',size=14)
                plt.title('FOLD %i - Image Size %i, EfficientNet B%i, inc2019=%i, inc2018=%i'%
                        (fold+1,IMG_SIZES[fold],EFF_NETS[fold],INC2019[fold],INC2018[fold]),size=18)
                plt.legend(loc=3)
                plt.show()  
    ######################################### 

# COMPUTE OVERALL OOF AUC
oof = np.concatenate(oof_pred)
true = np.concatenate(oof_tar)
names = np.concatenate(oof_names)
folds = np.concatenate(oof_folds)
auc = roc_auc_score(true, oof)
print('Overall OOF AUC with TTA = %.3f'%auc)


Range 0 (array([ 0,  2,  3,  5,  6,  8,  9, 10, 11, 12, 13, 14]), array([1, 4, 7]))


INFO:tensorflow:Initializing the TPU system: grpc://10.11.163.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.11.163.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


#########################
#### FOLD 1
#### Image Size 384 with EfficientNet B6 and batch_size 64
#### Using 2019 external data
['gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train00-2182.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train02-2193.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train03-2182.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train05-2171.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train06-2175.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train08-2177.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train09-2178.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train10-2174.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train11-2176.tfrec', 'gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4/train12-2198.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


733/733 [==============================] - 292s 399ms/step - auc: 0.7409 - loss: 0.3562 - val_auc: 0.7762 - val_loss: 0.1256 - lr: 1.0000e-05
Epoch 2/20
445/733 [=================>............] - ETA: 1:39 - auc: 0.8243 - loss: 0.2996

In [ ]:
# SAVE OOF TO DISK
df_oof = pd.DataFrame(dict(image_name=names, target=true, pred=oof, fold=folds))
df_oof.to_csv('oof.csv',index=False)
df_oof.head()

ds = get_dataset(files_test, augment=False, repeat=False, dim=IMG_SIZES[fold], labeled=False, return_image_names=True)
image_names = np.array([img_name.numpy().decode("utf-8") for img, img_name in iter(ds.unbatch())])

submission = pd.DataFrame(dict(image_name=image_names, target=preds[:,0]))
submission = submission.sort_values('image_name') 
submission.to_csv('submission.csv', index=False)
# submission.head()

In [ ]:
# submission.to_csv('submission.csv', index=False)
sub_file_name = 'submissionFAST_RUN-{}.csv'.format(SEED)

submission.to_csv(sub_file_name, index=False)
submission.head()

In [ ]:
from google.colab import files
# files.download('submission.csv')
# files.download(sub_file_name)
# files.download('oof.csv') 
print(sub_file_name)

In [ ]:
# submission.to_csv('submission.csv', index=False)
files.download('submission.csv')

In [ ]:
# files.download('oof.csv') 

In [ ]:
end = time.time()
use_time = end - start
print(end)
print('The total time is {} minutes'.format(round(use_time/60,1)))

In [ ]:
print(time.localtime(time.time()))

In [ ]:
#SIIM_para['run_time'] = round(use_time/60,1)
print('SIIM_parameters: {}'.format(SIIM_para))